In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading the dataset using pandas
heart_dataset = pd.read_csv("/kaggle/input/heart-disease-dataset/heart.csv")
print (heart_dataset.head())
print (heart_dataset.shape)

In [ ]:
#dividing deep inside the data
heart_dataset.dtypes

In [ ]:
heart_dataset.describe()

In [ ]:
#Making function to see all different types of values
def print_unique_values(df):
    for column in df:
        print( f'{column} : {df[column].unique()}')
        
print_unique_values(heart_dataset)

In [ ]:
#using sklearn to get everything between 0 and 1
from sklearn.preprocessing import MinMaxScaler
dataset = heart_dataset.copy()
scaler = MinMaxScaler()

data = dataset.drop("target", axis = "columns")
target = dataset["target"]

data = scaler.fit_transform(data)


In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data,target,test_size = 0.2,random_state =42)

print (len(data_train))

In [ ]:

data_train, data_valid, target_train, target_valid = train_test_split(data, target, test_size=0.20, shuffle= True)
print("The shape of X_train is      ", data_train.shape)
print("The shape of X_test is       ",data_test.shape)
print("The shape of y_train is      ",target_train.shape)
print("The shape of y_test is       ",target_test.shape)
print("The shape of Y_valid is      ", target_valid.shape)
print("The shape of X_valid is       ",data_valid.shape)


In [ ]:
#making model ANN using tensorflow
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Dense(150, input_shape = (820,13), activation = "relu"))#hidden layer with 300 neurons with relu activation15
model.add(keras.layers.Dense(75, activation = "relu")) #hidden layer with 100 neurons with relu activation
model.add(keras.layers.Dense(1, activation = "sigmoid"))


In [ ]:
#model summary 
model.summary()

In [ ]:
#logs for tensorboard 
import os

root_logdir = os.path.join(os.curdir, "my_logs")
print(os.path)
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
print(run_logdir)

In [ ]:
#optimizer Adam with particular hyperparameter 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001)

In [ ]:
#compiling model
model.compile(optimizer=optimizer,
             loss = "binary_crossentropy",
             metrics= ["accuracy"])
early_stopping_nn = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_nn = keras.callbacks.TensorBoard( log_dir = run_logdir, histogram_freq=1)
heart_model = model.fit(data_train, target_train, epochs= 100, validation_data = (data_valid,  target_valid), callbacks= [early_stopping_nn, tensorboard_nn])

In [ ]:
#plot graphs from model history
pd.DataFrame(heart_model.history).plot (figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
#evaluating on test dataset
model.evaluate(data_test, target_test)